In [26]:
import pandas as pd

df_test = pd.read_csv("Data_test.csv") # does not have asnwer metrics (`scores` in this case)
df_train = pd.read_csv("Data_train.csv")

df_train.head()

,type,group,education,meal,preparation course,score-1,score-2,score-3
0,wild,group B,some high school,standard,completed,63,67,67
1,domestic,group C,bachelor's degree,standard,none,67,69,75
2,domestic,group C,some college,standard,completed,69,90,88
3,domestic,group B,some high school,standard,none,62,64,66
4,wild,group E,some college,reduced,none,93,90,83


In [27]:
df_train.describe()

,score-1,score-2,score-3
count,700.000000,700.000000,700.000000
mean,66.028571,69.038571,67.828571
std,15.140138,14.589664,15.152323
min,0.000000,17.000000,10.000000
25%,56.000000,59.000000,57.000000
50%,66.000000,70.000000,68.000000
75%,77.000000,79.000000,78.000000
max,100.000000,100.000000,100.000000


In [28]:
df_train.shape

(700, 8)

In [29]:
df_train.describe(include='object') # включаем категориальные признаки
# видим, что в колонке group не достает 20 значений

,type,group,education,meal,preparation course
count,700,680,700,700,700
unique,2,5,6,2,2
top,domestic,group C,associate's degree,standard,none
freq,357,207,154,444,442


In [30]:
df_train['group'].fillna('Unknown', inplace=True) # заменяем NaN значения
df_train['score-1'].fillna(df_train['score-1'].mean()) # заменить все NaN значения на средние по столбцу
df_train.describe(include='object')

,type,group,education,meal,preparation course
count,700,700,700,700,700
unique,2,6,6,2,2
top,domestic,group C,associate's degree,standard,none
freq,357,207,154,444,442


 Кошка прошла полосу препятствий по мнению судьи, если он поставил ей больше 50 баллов. Кошка считается прошедшей полосу препятствий, если все судьи поставили ей больше 50 баллов. В df_train создайте колонку 'Pass' и запишите в неё 1, если кошка прошла полосу препятствий, и 0 иначе. В ответ запишите, сколько кошек из df_train не прошли полосу препятствий.

In [31]:
df_train['Pass'] = (df_train['score-1'] > 50) & (df_train['score-2'] > 50) & (df_train['score-3'] > 50)

df_train['Pass'] = df_train['Pass'].astype(int)
len(df_train[df_train['Pass'] == 0])

145

1) Среди всех диких кошек найдите долю кошек, прошедших полосу препятствий. Такую же долю рассчитайте для домашних кошек. В ответе укажите модуль разности этих долей. Ответ округлите до сотых.

In [32]:
wild = df_train[df_train['type'] == 'wild']
domestic = df_train[df_train['type'] == 'domestic']

wild_passed = wild[wild['Pass'] == 1]
wild_passed_val = len(wild_passed) / len(wild)

domestic_passed = domestic[domestic['Pass'] == 1]
domestic_passed_val = len(domestic_passed) / len(domestic)

abs(domestic_passed_val - wild_passed_val)

0.022580460755730947

2) Сколько кошек среди не прошедших полосу препятствий имели инструктора с уровнем образования "high school"?

In [33]:
not_passed = df_train[(df_train['education'] == 'high school') & (df_train['Pass'] == 0)]
len(not_passed)

35

3) Сколько диких кошек среди прошедших полосу препятствий не проходили специальный курс подготовки?

In [34]:
len(wild[(wild['preparation course'] != 'completed') & (wild['Pass'] == 1)])

152

4) Чему равна медиана баллов, выставленных первым судьей?


In [35]:
df_train['score-1'].median()

66.0

a) (0.3 баллa). Далее используйте только категориальные столбцы. Закодируйте их с помощью One-hot encoding с учетом того, что мы не хотим получить мультиколлинеарности в новых данных. Сколько получилось числовых столбцов из исходных категориальных? Кодируйте и df_train, и df_test.

In [36]:
df_train.columns

Index(['type', 'group', 'education', 'meal', 'preparation course', 'score-1',
       'score-2', 'score-3', 'Pass'],
      dtype='object')

In [37]:
df_new = pd.get_dummies(df_train, columns=['type', 'group', 'education', 'meal', 'preparation course'], drop_first=True)

df_new.shape

(700, 17)

б) (0.4 балла). Попытаемся по характеристикам кошки (бывшие категориальные, а теперь - числовые столбцы) предсказать, прошла она полосу препятствий или нет.


Сформируйте из df_train матрицу объект-признак X и вектор ответов y.

In [38]:
X = df_new.drop('Pass', axis=1)
y = df_new['Pass']

Обучите решающее дерево (DecisionTreeClassifier из библиотеки sklearn.tree) глубины 5 с энтропийным критерием информативности на закодированных в пункте а) тренировочных данных по кросс-валидации с тремя фолдами, метрика качества - roc-auc.  

Чему равен roc-auc, усредненный по фолдам? Ответ округлите до десятых.

In [39]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

tree = DecisionTreeClassifier(max_depth=5, criterion='entropy')
cross_val_score(tree, X, y, cv=3, scoring='roc_auc').mean()

0.9965986394557823

a) (0.25 балла). Подберите глубину решающего дерева (max_depth), перебирая глубину от 2 до 20 с шагом 1 и используя перебор по сетке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - roc-auc. В ответ запишите наилучшее среди искомых значение max_depth.

In [40]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# Предположим, df_new уже подготовлен и содержит закодированные числовые столбцы и целевой столбец 'Pass'
# Создание матрицы объект-признак и вектора ответов
X = df_new.drop('Pass', axis=1)
y = df_new['Pass']

# Создание решающего дерева
tree = DecisionTreeClassifier(criterion='entropy')

# Задание диапазона глубины дерева от 2 до 20
param_grid = {'max_depth': range(2, 21)}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=3, scoring='roc_auc')

# Обучение GridSearchCV
grid_search.fit(X, y)

# Получение наилучшего значения max_depth
best_max_depth = grid_search.best_params_['max_depth']

print(f"Наилучшее значение max_depth: {best_max_depth}")

Наилучшее значение max_depth: 4


б) (0.5 балла). Добавьте к данным новый признак cat_bio, содержащий в качестве значений пары значений из столбца type и столбца group. Например, если кошка имеет type='wild' и  group='group B', то в cat_bio будет стоять строка '(wild, group B)'. Примените OneHotEncoding (с учетом того, что мы не хотим получить мультиколлинеарности в новых данных) к столбцам 'cat_bio', 'education', 'meal', 'preparation course', а затем обучите решающее дерево глубины 5 с энтропийным критерием информативности на полученных после кодирования данных. Чему равен roc-auc? Ответ округлите до сотых.

In [41]:
df_new2 = df_train.copy()
df_new2['cat_bio'] = '(' + df_new2['type'] + ', ' + df_new2['group'] + ')'
df_new2 = pd.get_dummies(df_new2, columns=['cat_bio', 'education', 'meal', 'preparation course'], drop_first=True)
df_new2 = df_new2.drop(['type', 'group'], axis=1)

X = df_new2.drop('Pass', axis=1)
y = df_new2['Pass']

tree = DecisionTreeClassifier(max_depth=5, criterion='entropy')
cross_val_score(tree, X, y, cv=3, scoring='roc_auc').mean()

0.9965986394557823